In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from xgboost import XGBRegressor
import pickle   
import json

In [2]:
rootDir = ""
# import os
# rootDir = os.getcwd()+"/mysite/"

def getJSON(csvFile, index=0):
    df = pd.read_csv(csvFile)
    print(df.iloc[index].to_json())
    
def dummy_row(df):
    df2 = pd.DataFrame()
    for k in df:
        df2[k] = [0]
    df2.to_csv(rootDir+"dummy_row.csv", index=False)
    print(df.iloc[0])
def getDF(d1):
    d2 = {}
    for k in d1:
        v = []
        v.append(d1[k])
        d2[k]= v       
    return pd.DataFrame(d2) 
def from_dummy_row(user_df):
    import pandas as pd            
    df = pd.read_csv(rootDir+"dummy_row.csv")
    for k in user_df:
        df[k] =user_df[k]
    print(df.info())
    return df

In [3]:
def pre_proc(df):
    df.drop(["Province/State","Last Update","ObservationDate"],axis=1,inplace =True)    
    df = pd.get_dummies(df)
    return df

In [4]:
def training():
    df = pd.read_csv("covid_19_data.csv",parse_dates=["ObservationDate","Last Update"])    
    df = pre_proc(df)
    X = df.drop("Deaths", axis=1)
    dummy_row(X)
    y=df["Deaths"]
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=11) 
    model = RandomForestRegressor(random_state=11)
#     model = XGBRegressor(n_estimators=100)
    model.fit(X_train, y_train)
    with open("corna.pkl", "wb") as f1:
        pickle.dump(model, f1)    
    y_pred = model.predict(X_test)
    err = mean_squared_error(y_test, y_pred)
    print(err)
    print(model.score(X_test, y_test))
    from math import sqrt
    err2 = sqrt(err)
    print(err2)

In [5]:
# training()

In [6]:
def pred(jsonData):
#         x = json.loads(jsonData)    
#       df = json_normalize(x) #normalize dont create column for null values
        x= jsonData
        df = getDF(x)
#       tmp = df.iloc[0]
#       tmp.to_csv("tmp2.csv")    
        df = pre_proc(df)
        df = from_dummy_row(df)
        with open("corna.pkl", "rb") as f1:
            model = pickle.load(f1)    
        if "Deaths" in df:
            df.drop("Deaths", inplace=True, axis=1)
        y = model.predict(df)
        print(y)
        return y
    
# pred('{"SNo":2487,"ObservationDate":"02\/27\/2020","Province\/State":"Anhui","Country\/Region":"Mainland China","Last Update":"2020-02-27T14:13:06","Confirmed":989,"Deaths":6,"Recovered":792}')

In [7]:
# getJSON("covid_19_data.csv", 2486)